In [1]:
from pyzotero import zotero
import os
import sys
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm.notebook import tqdm
tqdm.pandas()

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from code_utils.zotero import get_data_from_zotero
from code_utils.glutton import get_doi_glutton
from code_utils.utils import get_doi_cleaned,aplatir,wg_chap_to_dict,get_year_ipbes,check_doi_glutton
from code_utils.pickle import load_cache,write_cache
from code_utils.enriching_data_OpenAlex import get_countries_concepts_sdg,get_open_alex_data,get_open_alex_data_not_in_references

In [2]:
cached_openalex_data_ipbes = {}
cached_openalex_data_not_ipbes = {}

In [ ]:
try:
    cached_openalex_data_ipbes = load_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_ipbes.pkl')
    cached_openalex_data_not_ipbes = load_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipbes.pkl')
except:
    #write_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_ipbes.pkl')
    #write_cache(cached_openalex_data_not_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipbes.pkl')
    print('oups')

Read IPBES references in zotero files

In [ ]:
ZOTERO_KEY = os.getenv('ZOTERO_KEY')
IPBES_ZOTERO_ID = '2333077'
COLLECTION_IDS = ['8DQ8YFJI','JR5LKU4U','LBFNF62B','7IMYH9U3','DRZLUY9K','KTPTCAVF','UWNSUNXQ','BJGWRJAK','JT8FJXP6']
CHAPTER_IDS=['1','2.1','2.2','2.3','3','4','5','6','glossary']
ALL_COLLECTIONS={}

In [ ]:
ALL_COLLECTIONS=get_data_from_zotero(ZOTERO_KEY,IPBES_ZOTERO_ID,COLLECTION_IDS,CHAPTER_IDS,ALL_COLLECTIONS)

In [ ]:
df_ipbes_chap={}
for i in range(len(COLLECTION_IDS)):
    df_ipbes_chap[CHAPTER_IDS[i]]=pd.DataFrame(list(pd.DataFrame(ALL_COLLECTIONS[f'collection_chapter_{CHAPTER_IDS[i]}']).data))[['DOI','title','date','creators','tags']]
    df_ipbes_chap[CHAPTER_IDS[i]]['chap']=f'chap_{CHAPTER_IDS[i]}'
    print(f'{len(df_ipbes_chap[CHAPTER_IDS[i]][~pd.isna(df_ipbes_chap[CHAPTER_IDS[i]].DOI)])/len(df_ipbes_chap[CHAPTER_IDS[i]])*100} % doi available for chapter {CHAPTER_IDS[i]}')

Clean and enrich data 

In [ ]:
df_ipbes=pd.concat(list(df_ipbes_chap.values())).reset_index()
del df_ipbes['index']

In [ ]:
len(df_ipbes[~pd.isna(df_ipbes.DOI)])/len(df_ipbes)*100

In [ ]:
df_ipbes.DOI.drop_duplicates()

In [ ]:
get_doi_glutton(df_ipbes.iloc[1,:])

In [ ]:
df_ipbes.loc[pd.isna(df_ipbes.DOI),'DOI']=df_ipbes.loc[pd.isna(df_ipbes.DOI),:].progress_apply(get_doi_glutton, axis=1)

In [ ]:
len(df_ipbes[~pd.isna(df_ipbes.DOI)])/len(df_ipbes)*100

In [ ]:
df_ipbes.loc[:,'DOI']=df_ipbes.loc[:,'DOI'].apply(lambda x: get_doi_cleaned(x))
df_ipbes.loc[:,'DOI']=df_ipbes.loc[:,'DOI'].apply(lambda x: None if str(x)[:4]=='http' else x)
df_ipbes['freq']=1
df_ipbes=df_ipbes.rename(columns={'DOI':'doi'})

In [ ]:
df_ipbes.to_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes.json', orient='records')

Enriching data through OpenAlex API integration

In [ ]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes.json', orient='records')

In [ ]:
df_unique_doi=df_ipbes.groupby(by=['doi'], dropna=True).agg({'chap': lambda x: list(x), 'freq': 'sum'})
df_unique_doi=df_unique_doi.reset_index()

In [ ]:
#get data_OpenAlex from the API 
df_unique_doi.apply(lambda row: get_open_alex_data(cached_openalex_data_ipbes,row['doi']), axis= 1)

In [ ]:
len(cached_openalex_data_ipbes)

In [ ]:
write_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_ipbes.pkl')

In [ ]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes.json')

In [ ]:
df_unique_doi=df_ipbes['doi'].drop_duplicates().dropna()
df_unique_doi=pd.DataFrame({'doi':list(df_unique_doi)})

In [ ]:
countries_list = []
concepts_list = []
sdg_list = []
topics_list = []
is_OA_available_list=[]
title_list=[]
year_list=[]
names_list=[]
rors_list=[]
institutions_names_list=[]
locations_names_list=[]
locations_ids_list=[]

for i,row in df_unique_doi.iterrows():
    countries,concepts,sdg,year,topics,doi,bool_OA,title,name,rors,institutions_names,locations_names,locations_ids=get_countries_concepts_sdg(cached_openalex_data_ipbes,row,ipcc=True,i=0)

    countries_list.append(countries)
    concepts_list.append(concepts)
    sdg_list.append(sdg)
    topics_list.append(topics)  
    is_OA_available_list.append(bool_OA)
    title_list.append(title)
    year_list.append(year)
    names_list.append(name)
    rors_list.append(rors)
    institutions_names_list.append(institutions_names)
    locations_names_list.append(locations_names)
    locations_ids_list.append(locations_ids)


df_unique_doi['countries'] = countries_list
df_unique_doi['concepts'] = concepts_list
df_unique_doi['sdg'] = sdg_list
df_unique_doi['topics'] = topics_list
df_unique_doi['is_OA_available'] = is_OA_available_list
df_unique_doi['year_OA'] = year_list
df_unique_doi['authors_name'] = names_list
df_unique_doi['rors'] = rors_list
df_unique_doi['institutions_names'] = institutions_names_list
df_unique_doi['locations_names'] = locations_names_list
df_unique_doi['locations_id'] = locations_ids_list

In [ ]:
df_unique_doi.is_OA_available.value_counts()

In [ ]:
df_unique_doi=df_unique_doi[df_unique_doi.is_OA_available]

In [ ]:
df_unique_doi=df_unique_doi.reset_index()
del df_unique_doi['index']

In [ ]:
df_unique_doi.to_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_OA.json', orient='records')

Ipbes references data

In [ ]:
df_unique_doi=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_OA.json', orient='records')

In [ ]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes.json')

In [ ]:
df_ipbes=pd.merge(df_ipbes[['doi', 'title', 'date', 'creators', 'tags']], df_unique_doi, on='doi', how='inner')

In [ ]:
df_ipbes['year']=df_ipbes['date'].apply(lambda x: get_year_ipbes(x) if pd.isna(x)==False else None)

In [ ]:
df_ipbes['check']=df_ipbes.apply(lambda row: check_doi_glutton(row), axis=1)
df_ipbes.check.value_counts()

In [ ]:
df_ipbes[df_ipbes.check==False]

In [ ]:
df_ipbes=df_ipbes[(df_ipbes.check) & (pd.isna(df_ipbes['is_OA_available'])==False)]

In [ ]:
del df_ipbes['check']

In [ ]:
df_ipbes.loc[:,'countries']=df_ipbes.loc[:,'countries'].apply(lambda x: str(x).replace('nan','None').replace('[]','None').replace('[None]','None') if ((str(x)=='nan')|(str(x)=='[None]')|((str(x)=='[]'))) else x )
df_ipbes.loc[:,'countries']=df_ipbes.loc[:,'countries'].apply(lambda x: ['None'] if x=='None' else x)

In [ ]:
df_ipbes.to_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipbes\\data_ipbes.jsonl', orient='records',lines=True)

Find the data for constructing the learning model ( IPBES related or not )

In [3]:
df_ipbes=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipbes\\data_ipbes.jsonl', lines= True).dropna(subset=['year'])

In [4]:
df_ipbes['year'] = pd.to_numeric(df_ipbes['year'], errors='coerce')
df_ipbes=df_ipbes.dropna(subset=['year','doi', 'title','topics','locations_names','locations_id']).drop_duplicates(subset=['doi'])
year=df_ipbes.drop_duplicates(subset='doi').dropna(subset=['doi'])['year'].dropna().sort_values().apply(lambda x: int(x))
year_counts = pd.Series(list(year)).value_counts().to_dict()

In [5]:
dois=list(df_ipbes.doi.dropna().drop_duplicates())

In [6]:
sum(list(year_counts.values()))

5531

In [7]:
year_counts_not_ipbes={}

In [8]:
for year in list(year_counts.keys()):   
    cached_openalex_data_not_ipbes[year]=[]
    year_counts_not_ipbes[year]=0
    while year_counts_not_ipbes[year]<year_counts[year]:
        get_open_alex_data_not_in_references(dois,cached_openalex_data_not_ipbes,year_counts,year_counts_not_ipbes,year)
    cached_openalex_data_not_ipbes[year]=cached_openalex_data_not_ipbes[year][:year_counts[year]]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443
DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:2016&sample=200&per-page=200 HTTP/1.1" 200 297395
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443


plus que 624 publications pour completer l'année 2016


DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:2016&sample=200&per-page=200 HTTP/1.1" 200 290705
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443


plus que 513 publications pour completer l'année 2016


DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:2016&sample=200&per-page=200 HTTP/1.1" 200 311560
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443


...

plus que 1 publications pour completer l'année 1976


DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:1954&sample=200&per-page=200 HTTP/1.1" 200 156303
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443


plus que 1 publications pour completer l'année 1954


DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:1936&sample=200&per-page=200 HTTP/1.1" 200 138554


plus que 1 publications pour completer l'année 1936


In [9]:
sum(list(year_counts.values()))

5531

In [10]:
len(aplatir(list(cached_openalex_data_not_ipbes.values())))


5531

In [11]:
len(cached_openalex_data_not_ipbes)

58

In [12]:
write_cache(cached_openalex_data_not_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipbes.pkl')

In [13]:
df_not_ipbes=pd.DataFrame()
countries_list = []
concepts_list = []
sdg_list = []
topics_list = []
is_OA_available_list=[]
title_list=[]
year_list=[]
names_list=[]
rors_list=[]
institutions_names_list=[]
locations_names_list=[]
locations_ids_list=[]
doi_list=[]

for k in range(len(aplatir(list(cached_openalex_data_not_ipbes.values())))):
    countries,concepts,sdg,year,topics,doi,bool_OA,title,name,rors,institutions_names,locations_names,locations_ids=get_countries_concepts_sdg(cached_openalex_data=aplatir(list(cached_openalex_data_not_ipbes.values())),ipcc=False,i=k)

    countries_list.append(countries)
    concepts_list.append(concepts)
    sdg_list.append(sdg)
    topics_list.append(topics)  
    is_OA_available_list.append(bool_OA)
    title_list.append(title)
    year_list.append(year)
    names_list.append(name)
    rors_list.append(rors)
    institutions_names_list.append(institutions_names)
    locations_names_list.append(locations_names)
    locations_ids_list.append(locations_ids)
    doi_list.append(doi)


df_not_ipbes['doi'] = doi_list
df_not_ipbes['title'] = title_list
df_not_ipbes['countries'] = countries_list
df_not_ipbes['concepts'] = concepts_list
df_not_ipbes['sdg'] = sdg_list
df_not_ipbes['topics'] = topics_list
df_not_ipbes['is_OA_available'] = is_OA_available_list
df_not_ipbes['year'] = year_list
df_not_ipbes['authors_name'] = names_list
df_not_ipbes['rors'] = rors_list
df_not_ipbes['institutions_names'] = institutions_names_list
df_not_ipbes['locations_names'] = locations_names_list
df_not_ipbes['locations_id'] = locations_ids_list

In [14]:
df_not_ipbes

,doi,title,countries,concepts,sdg,topics,is_OA_available,year,authors_name,rors,institutions_names,locations_names,locations_id
0,https://doi.org/10.26499/ttbng.v4i1.3,DISTRIBUSI FONEM BAHASA DI PULAU SAPARUA: DATA...,[],"[Islam, Linguistics, Humanities, Theology, Art...","[{'id': '4', 'name': 'Quality education'}]","[Linguistics and Language Analysis, English La...",True,2016,"[(Erniati Erniati, [])]",[],[],TOTOBUANG,2339-1154
1,https://doi.org/10.1007/s38313-016-0096-0,Active High-pressure Impactor Flexible Oil Sep...,[DE],"[Diesel fuel, Automotive engineering, Separati...","[{'id': '7', 'name': 'Affordable and clean ene...","[Tribology and Lubrication Engineering, Lubric...",True,2016,"[(Stefan Ruppel, [DE]), (Volker Kirschner, [DE...","[(https://ror.org/0117jxy09, DE), (https://ror...","[(Springer Nature (Germany), DE), (Springer Na...",MTZ worldwide,2192-9114
2,https://doi.org/10.4172/2470-0886.1000112,Demystifying Apples in Health and Dentistry,[],"[Enamel paint, Dentistry, Food science, Medici...",None,"[Dental Erosion and Treatment, Oral Health Pat...",True,2016,"[(Louis Z.G. Touyz, [])]",[],[],Dental Health Current Research,2470-0886
3,https://doi.org/10.3847/0004-6256/151/6/158,TRIPPy: TRAILED IMAGE PHOTOMETRY IN PYTHON,"[US, TW, CA, GB, FR]","[Photometry (optics), Point spread function, P...",None,"[Advanced Image Processing Techniques, Galaxie...",True,2016,"[(Wesley C. Fraser, [GB]), (Mike Alexandersen,...","[(https://ror.org/00hswnk62, GB), (https://ror...","[(Queen's University Belfast, GB), (National T...",The Astronomical Journal,0004-6256
4,https://doi.org/10.3109/01612840.2015.1111475,"Yawning, Why and When?",[US],"[Psychology, MEDLINE, Medicine, Political scie...",None,"[Human-Animal Interaction Studies, Primate Beh...",True,2016,"[(Jacquelyn H. Flaskerud, [US])]","[(https://ror.org/046rm7j60, US)]","[(University of California, Los Angeles, US)]",Issues in Mental Health Nursing,0161-2840
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5526,https://doi.org/10.7589/0090-3558-10.2.111,EMACIATION AND ENTERITIS OF COTTONTAIL RABBITS...,[US],"[Emaciation, Biology, Enteritis, Anatomy, Zool...","[{'id': '15', 'name': 'Life on land'}]","[Rabbits: Nutrition, Reproduction, Health, Hel...",True,1974,"[(Harry A. Jacobson, [US]), (R. L. Kirkpatrick...","[(https://ror.org/02smfhw86, US), (https://ror...","[(Virginia Tech, US), (Virginia Tech, US), (Vi...",Journal of Wildlife Diseases,0090-3558
5527,https://doi.org/10.1021/i260056a022,"The Role of Liquid Holdup, Effective Catalyst ...",[],"[Wetting, Icon, Citation, Computer science, Op...","[{'id': '6', 'name': 'Clean water and sanitati...",[Catalysis and Hydrodesulfurization Studies],True,1975,"[(Angelo A. Montagna, []), (Yatish T. Shah, [])]",[],[],Industrial & Engineering Chemistry Process Des...,0196-4305
5528,https://doi.org/10.1080/05695557608975071,A Branch and Bound Algorithm for Use in Resour...,[US],"[Weighting, Resource allocation, Branch and bo...",None,"[Resource-Constrained Project Scheduling, Sche...",True,1976,"[(Peter C. Katsoulas, [US]), (Randall P. Sadow...","[(https://ror.org/0072zz521, US)]","[(University of Massachusetts Amherst, US)]",A I I E Transactions,0569-5554
5529,https://doi.org/10.1007/bf02247002,Application of particle dynamics to derive a g...,[GB],"[Zonal and meridional, Equator, Streamlines, s...","[{'id': '13', 'name': 'Climate action'}]","[Tropical and Extratropical Cyclones Research,...",True,1954,"[(A. H. Gordon, [GB]), (Patrick Shaw, [GB])]","[(https://ror.org/01ch2yn61, GB), (https://ror...","[(Met Office, GB), (Met Office, GB)]",Archives for Meteorology Geophysics and Biocli...,0066-6416


In [15]:
df_not_ipbes.to_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipbes\\data_not_ipbes.jsonl', orient='records', lines= True)